<a href="https://colab.research.google.com/github/ctrivino1/Anomoly_detection/blob/main/Building_A_Full_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random

class NeuralNetwork:
  def init(self, inputs, outputs, hidden_layers=1, neurons_per_layer=4):
    self.inputs = inputs
    self.outputs = outputs
    self.hidden_layers = hidden_layers
    self.neurons_per_layer = neurons_per_layer

      # Initialize weights and biases
    self.weights = []
    self.biases = []
    for i in range(hidden_layers + 1):
        if i == 0:
            layer_inputs = inputs.shape[1]
        else:
            layer_inputs = neurons_per_layer
        if i == hidden_layers:
            layer_outputs = outputs.shape[1]
        else:
            layer_outputs = neurons_per_layer
        self.weights.append(np.random.rand(layer_inputs, layer_outputs))
        self.biases.append(np.zeros((1, layer_outputs)))

  def sigmoid(self, x):
    return 1/(1 + np.exp(-x))

  def sigmoid_derivative(self, x):
    return x * (1 - x)

  def feedforward(self, inputs):
    current_input = inputs
    for i in range(self.hidden_layers + 1):
        current_output = self.sigmoid(np.dot(current_input, self.weights[i]) + self.biases[i])
        current_input = current_output
    return current_output

  def backpropagate(self, inputs, targets, learning_rate):
    # Feedforward
    current_input = inputs
    activations = [inputs]
    for i in range(self.hidden_layers + 1):
        current_output = self.sigmoid(np.dot(current_input, self.weights[i]) + self.biases[i])
        activations.append(current_output)
        current_input = current_output
    
    # Backpropagation
        delta_output = (targets - activations[-1]) * self.sigmoid_derivative(activations[-1])
        for i in range(self.hidden_layers, 0, -1):
            delta = np.dot(delta_output, self.weights[i].T) * self.sigmoid_derivative(activations[i])
            self.weights[i] += learning_rate * np.dot(activations[i].T, delta_output)
            self.biases[i] += learning_rate * np.sum(delta_output, axis=0, keepdims=True)
            delta_output = delta
        self.weights[0] += learning_rate * np.dot(inputs.T, delta_output)
        self.biases[0] += learning_rate * np.sum(delta_output, axis=0, keepdims=True)

  def train(self, inputs, targets, learning_rate, epochs, test_inputs, test_targets):
        for i in range(epochs):
            self.backpropagate(inputs, targets, learning_rate)
            if i % 100 == 0:
                predictions = self.feedforward(test_inputs)
                accuracy = np.mean(np.round(predictions) == test_targets)
                print("Epoch %d, loss: %f, accuracy: %f" % (i, np.mean(np.square(targets - self.feedforward(inputs))), accuracy))

  def save_weights_biases(self, filename):
        np.savez(filename, *self.weights, *self.biases)
    
  def load_weights_biases(self, filename):
        data = np.load(filename)
        for i in range(len(self.weights)):
            self.weights[i] = data['arr_%d' % (2 * i)]
            self.biases[i] = data['arr_%d' % (2 * i + 1)]

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your data into a Pandas DataFrame
# Define the dataframe columns
columns = ['input1', 'input2', 'input3', 'target']

# Create a list of artificial data
data = []
for i in range(100):
    input1 = random.uniform(0, 1)
    input2 = random.uniform(0, 1)
    input3 = random.uniform(0, 1)
    target = random.uniform(0, 1)
    data.append([input1, input2, input3, target])

# Create the dataframe from the list
df = pd.DataFrame(data, columns=columns)


# Split the data into inputs and targets
inputs = df.iloc[:, :-1].values
targets = df.iloc[:, -1].values

# Convert the targets to one-hot encoding
targets = np.array([[1, 0] if x == 0 else [0, 1] for x in targets])

# Create the neural network
nn = NeuralNetwork()
nn.init(inputs, targets)

# Train the network
nn.train(inputs, targets, learning_rate=0.1, epochs=1000, test_inputs=inputs, test_targets=targets)

# Test the network
predictions = nn.feedforward(inputs)
predictions = np.round(predictions)
accuracy = np.mean(predictions == targets)
print("Accuracy:", accuracy)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# the neural network in the code is trying to predict the class of the target variable, which is binary (0 or 1).
# The targets are one-hot encoded and the network is trying to predict the class with the highest probability. 
# The accuracy is then calculated based on the comparison between the predicted classes and the actual classes.

# Load your data into a Pandas DataFrame
# Define the dataframe columns
columns = ['input1', 'input2', 'input3', 'target']

# Create a list of artificial data
data = []
for i in range(100):
    input1 = random.uniform(0, 1)
    input2 = random.uniform(0, 1)
    input3 = random.uniform(0, 1)
    target = random.uniform(0, 1)
    data.append([input1, input2, input3, target])

# Create the dataframe from the list
df = pd.DataFrame(data, columns=columns)


# Split the data into inputs and targets
inputs = df.iloc[:, :-1].values
targets = df.iloc[:, -1].values

# Convert the targets to one-hot encoding
targets = np.array([[1, 0, 0, 0] if x < 0.25 else [0, 1, 0, 0] if x < 0.5 else [0, 0, 1, 0] if x < 0.75 else [0, 0, 0, 1] for x in targets])

# Create the neural network
nn = NeuralNetwork()
nn.init(inputs, targets)

# Train the network
nn.train(inputs, targets, learning_rate=0.1, epochs=1000, test_inputs=inputs, test_targets=targets)

# Test the network
predictions = nn.feedforward(inputs)
predictions = np.round(predictions)
accuracy = np.mean(predictions == targets)
print("Accuracy:", accuracy)